In [14]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [15]:
# parametros cosecha
periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
lotes = []
penalizaciones = {
    'C1': [0.03, 0.02, 0.01, 0.005],
    'C2': [0.025, 0.012, 0.004, 0.002],
    'C3': [0.03, 0.02, 0.01, 0.005],
    'C4': [0.025, 0.012, 0.01, 0.002],
    'C5': [0.03, 0.02, 0.01, 0.005],
    'C6': [0.025, 0.012, 0.004, 0.002]
}

In [16]:
# Leo la base de datos y la guardo en una lista
c.execute("SELECT * FROM lotes")
lotes_obtenidos = c.fetchall()
lotes.append(lotes_obtenidos)
lotes = lotes[0]

In [17]:
#Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

# pasar lotes a lista de listas
lotes = [list(elem) for elem in lotes]
# Funcion que estima el dia de cosecha con media 0 y desv est 2
def dia_estimado():
        optimal_day = np.random.normal(0, 2)
        return int(round(optimal_day))
for lote in lotes:
    variacion_optimo = dia_estimado()
    lote[5]+=variacion_optimo
    lote.append(variacion_optimo)


In [18]:
# Lotes actualizados
print(lotes)

[[1, 'L_1_C4', 1, 'C4', 217.5, 116, 0.43, 0.39, 0.257, -2], [2, 'L_2_C4', 2, 'C4', 517.5, 121, 0.35, 0.44, 0.269, -2], [3, 'L_3_C6', 3, 'C6', 450.0, 80, 0.46, 0.38, 0.362, 1], [4, 'L_4_C4', 4, 'C4', 168.75, 126, 0.32, 0.51, 0.276, 4], [5, 'L_5_C6', 5, 'C6', 243.75, 104, 0.49, 0.6, 0.348, 0], [6, 'L_6_C3', 6, 'C3', 150.0, 95, 0.31, 0.56, 0.579, -1], [7, 'L_7_C5', 7, 'C5', 558.75, 109, 0.43, 0.39, 0.795, 0], [8, 'L_8_C6', 8, 'C6', 442.5, 74, 0.38, 0.35, 0.36, -4], [9, 'L_9_C1', 9, 'C1', 300.0, 124, 0.36, 0.35, 0.528, -1], [10, 'L_10_C5', 10, 'C5', 108.75, 79, 0.5, 0.56, 0.806, 0], [11, 'L_11_C1', 11, 'C1', 555.0, 99, 0.31, 0.53, 0.557, -1], [12, 'L_12_C1', 12, 'C1', 412.5, 124, 0.41, 0.33, 0.516, 0], [13, 'L_13_C5', 13, 'C5', 547.5, 89, 0.5, 0.48, 0.811, 0], [14, 'L_14_C2', 14, 'C2', 498.75, 113, 0.44, 0.52, 0.416, -3], [15, 'L_15_C1', 15, 'C1', 101.25, 127, 0.31, 0.48, 0.554, 0], [16, 'L_16_C6', 16, 'C6', 502.5, 97, 0.47, 0.34, 0.361, 0], [17, 'L_17_C5', 17, 'C5', 536.25, 112, 0.45, 0.3

In [19]:
# Función para predecir el clima con markov
def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
    clima_prediccion = []  # Lista para almacenar la predicción
    tipo_3 = []
    # Inicializar el primer día como seco
    clima_prediccion.append(0)
    tipo_3.append(0)
    # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
    for _ in range(7):
        if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
            clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
            tipo_3.append(0)
        else:  # Si el día anterior fue lluvioso
            clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
            if clima_prediccion[-1] == 1:
                tipo_3.append(1 if random.random() <= 0.05 else 0)
    if sum(tipo_3) >= 1:
        perdida = True
    else:
        perdida = False
    return clima_prediccion, perdida

In [20]:
def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 7
    while index != 0:
        if index == 7:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 6:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad


In [21]:
# funcion que entrega el valor final de la calidad considerando la lluvia
def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

In [22]:
# Parametros para encontrar en la simulacion
calidades_lotes = []
lotes_perdidos = 0
lotes_obtenidos = []

In [23]:
# la simulación
index = 0
for t in range(50, 150):
    for lote in lotes:
        lote_id = lote[0]
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_dia_optimo = lote[5]
        lote_prob_seco_lluvia = lote[6]
        lote_prob_lluvia_lluvia = lote[7]
        lote_precio_usd = lote[8]
        lote_variacion = lote[9]
        if t == lote_dia_optimo - 8:
            lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
            if perdida == True:
                print("lote_perdido")
                lotes_perdidos += 1
                pass
            elif not perdida:
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lotes_obtenidos.append(lote)
                lotes_obtenidos[index].append(calidad_lote)
                index += 1
                

lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido


In [24]:
# Encontrar la info que queramos de esta etapa
print(lotes_perdidos)
print(calidades_lotes)
print(max(calidades_lotes))
print(min(calidades_lotes))
print(lotes_obtenidos)
print(len(lotes_obtenidos))

11
[0.9199050352517985, 0.93484429, 0.9573603061224489, 0.9760036224489796, 0.9421562614666864, 0.96515, 0.9411410381642219, 0.994011992, 0.9496173673469388, 0.957384481887755, 0.9751, 0.9701129776530611, 0.9533318367346938, 1.0, 0.9326904297448979, 0.9589382149999999, 0.97522412625, 0.9751, 0.9993877551020408, 0.9810817217203919, 0.9554984999999999, 0.9850178530612245, 0.9720570918367346, 0.9410212499999999, 0.992019984, 0.990035944032, 0.986506187162449, 0.98505, 0.9602257653061225, 0.9594114134693876, 0.9762286454791836, 0.976028269005102, 0.9273399999999999, 0.9672194843999999, 0.9629609492346938, 0.971869387755102, 0.9745539248979592, 0.91277, 0.9221072645408163, 0.9813957240816327, 0.978155512703616, 0.9548988444489795, 0.8443486744897958, 0.941117765, 0.9691181632653061, 0.9852190824489796, 0.970125, 0.9666273092054694, 0.9614487591836735, 0.9931965648979592, 0.9571884973469387, 0.9509682040816326, 0.9939713387755102, 0.9728112244897958, 0.9971853061224489, 0.962185598877551, 0.

In [25]:
# Guardar los lotes obtenidos en la base de datos
# Ejecutar solo una vez
import sqlite3



# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('etapa_1.db')

### AQUI SACAR # PARA QUE FUNCIONE ESTA CELDA Y SE PUEDA CORRER
cursor = conn.cursor()

# Crear una tabla
cursor.execute('''
CREATE TABLE IF NOT EXISTS lotes (
    lote_id INTEGER PRIMARY KEY,
    lote_cod TEXT,
    lote_numero INTEGER,
    lote_tipo_uva TEXT,
    lote_toneladas REAL,
    lote_dia_optimo INTEGER,
    lote_prob_seco_lluvia REAL,
    lote_prob_lluvia_lluvia REAL,
    lote_precio_usd REAL,
    lote_variacion INTEGER,
    lote_calidad REAL
)
''')

# Insertar datos en la tabla
for lote in lotes_obtenidos:
    cursor.execute('''
    INSERT INTO lotes (
        lote_id, lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, 
        lote_dia_optimo, lote_prob_seco_lluvia, lote_prob_lluvia_lluvia, 
        lote_precio_usd, lote_variacion, lote_calidad
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', lote)

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()
